# Speaker recognition using singing or speaking voice

This project aims at recognizing speakers through a joint representation learning of singing and speaking voices from them. 

In [4]:
import pandas as pd
import numpy as np
import torch
import json

### JukeBox dataset analysis

[Voxceleb Link](https://mm.kaist.ac.kr/datasets/voxceleb/#downloads)

In [5]:
metadata = pd.read_csv("./jukebox_metadata.csv")

#creating artist id and name pairs
idname_df = metadata[['artist_id', 'artist_name']]
ids = list(idname_df['artist_id'])
names = list(idname_df['artist_name'])

id_name_dict = {}

for i in range(0, len(ids)):
    
    if ids[i] in id_name_dict:
        continue
    else:
        id_name_dict[int(ids[i])] = names[i]

# with open("./jukebox_id_artist.json", "w+") as data:
#     json.dump(id_name_dict, data)

In [6]:
id_name_dict

{0: 'A-Lin',
 2: 'Adira',
 3: 'Adrian Minune',
 4: 'Afgan',
 6: 'Ai',
 9: 'Aiko',
 10: 'Aimer',
 13: 'Albert West',
 14: 'Alex Mica',
 15: 'Alexandra Stan',
 16: 'Alma',
 17: 'Alya',
 20: 'Amel Bent',
 21: 'Amália Rodrigues',
 22: 'Ana Gabriel',
 24: 'Anahí',
 27: 'Annabel',
 28: 'Anneke Grönloh',
 31: 'Antonio Aguilar',
 32: 'António Variações',
 33: 'Anuar Zain',
 35: 'Ari Lasso',
 36: 'Arun Thapa',
 37: 'Aruna Lama',
 40: 'Aya Nakamura',
 41: 'Ayu Ting Ting',
 43: 'Barbara',
 45: 'Beni',
 46: 'Bernadette',
 50: 'Boris',
 53: 'Bourvil',
 55: 'Brigitte Bardot',
 56: 'Broery Marantika',
 57: 'Bunga Citra Lestari',
 58: 'C. Jérôme',
 59: 'Cali',
 60: 'Camelia',
 62: 'Carla Bruni',
 63: 'Carla Morrison',
 64: 'Carlos',
 66: 'Caro Emerald',
 70: 'Centerfold',
 73: 'Charles Aznavour',
 74: 'Charles Trénet',
 75: 'Charlotte Gainsbourg',
 79: 'Christian Nodal',
 80: 'Christophe Maé',
 81: 'Chrisye',
 82: 'Cinta Laura',
 83: 'Cita Citata',
 85: 'Claudine Longet',
 89: 'Corina',
 90: 'Cornelis

### Jukebox and VoxCeleb common artists

In [7]:
#jukebox artists
with open("./jukebox_id_artist.json", 'r') as juke:
    jukebox = json.load(juke)

jukebox_arts = [j for i,j in jukebox.items()]
jukebox_arts = [i.replace(" ", "_").lower() for i in jukebox_arts]
print(jukebox_arts[:10])


#voxceleb artists
voxDf = pd.read_csv("./vox1_meta.csv", delimiter="\t")
vox_arts = list(voxDf["VGGFace1 ID"])
vox_arts = [k.lower() for k in vox_arts]
print(vox_arts[:10])

#commons
commons = list(set(jukebox_arts).intersection(set(vox_arts)))
# with open("commons_in_vox_and_jukebox.txt", 'w+') as commonData:

#     for artist in commons:
#         commonData.write(artist)
#         commonData.write("\n")



['a-lin', 'adira', 'adrian_minune', 'afgan', 'ai', 'aiko', 'aimer', 'albert_west', 'alex_mica', 'alexandra_stan']
['a.j._buckley', 'a.r._rahman', 'aamir_khan', 'aaron_tveit', 'aaron_yoo', 'abbie_cornish', 'abigail_breslin', 'abigail_spencer', 'adam_beach', 'adam_brody']


### housekeeping for arranging vox and juke data

In [8]:
jukemeta = pd.read_csv("jukebox_metadata.csv")
voxmeta = pd.read_csv("vox1_meta.csv", delimiter='\t')
voxmeta['VGGFace1 ID'] = voxmeta['VGGFace1 ID'].str.casefold()
jukemeta['artist_name'] = jukemeta['artist_name'].str.casefold()
jukemeta['artist_name'] = jukemeta['artist_name'].apply(lambda x: x.replace(" ", "_"))

In [9]:
### creating dictionary for voxid:artist pair

voxcommon = voxmeta[voxmeta['VGGFace1 ID'].isin(commons)]
voxid_celeb = {}
for i in range(0, len(voxcommon)):
    voxid_celeb[voxcommon.iloc[i]['VoxCeleb1 ID']]=voxcommon.iloc[i]['VGGFace1 ID']

# with open("./vox_id_celeb.json", 'w+') as voxidartist:
    
#     json.dump(voxid_celeb, voxidartist)

In [10]:
### creating dictionary for jukeID:artist pair
jukecommon = jukemeta[jukemeta['artist_name'].isin(commons)]
#print(list(jukecommon))

jukeid_celeb = {}
for c in commons:
    jukeid = str(jukecommon[jukecommon['artist_name'] == c]['artist_id'].unique()[0])
    jukeid_celeb[jukeid] = c

# with open("./juke_id_celeb.json", 'w+') as jukeidartist:
#     json.dump(jukeid_celeb, jukeidartist)



In [11]:
jukecommon[:10]

,Unnamed: 0,song_name,artist_id,artist_name,partition,language_label,gender_label,full_path
2411,2411,0_1.wav,75,charlotte_gainsbourg,AUXILIARY,French,female,AUXILIARY/75/0_1.wav
2412,2412,0_2.wav,75,charlotte_gainsbourg,AUXILIARY,French,female,AUXILIARY/75/0_2.wav
2413,2413,0_3.wav,75,charlotte_gainsbourg,AUXILIARY,French,female,AUXILIARY/75/0_3.wav
2414,2414,0_4.wav,75,charlotte_gainsbourg,AUXILIARY,French,female,AUXILIARY/75/0_4.wav
2415,2415,0_5.wav,75,charlotte_gainsbourg,AUXILIARY,French,female,AUXILIARY/75/0_5.wav
14825,14825,0_1.wav,359,amanda_seyfried,TRAIN,English,female,TRAIN/359/0_1.wav
14826,14826,0_2.wav,359,amanda_seyfried,TRAIN,English,female,TRAIN/359/0_2.wav
14827,14827,0_3.wav,359,amanda_seyfried,TRAIN,English,female,TRAIN/359/0_3.wav
14828,14828,0_4.wav,359,amanda_seyfried,TRAIN,English,female,TRAIN/359/0_4.wav
14829,14829,0_5.wav,359,amanda_seyfried,TRAIN,English,female,TRAIN/359/0_5.wav


### Metadata Creation


In [14]:
from src.utilities import Utilities

with open("juke_id_celeb_commons.json", 'r') as jk:
    juke = json.load(jk)

with open("vox_id_celeb_commons.json", 'r') as vo:
    vox = json.load(vo)

In [15]:
commons

['troye_sivan',
 'lea_michele',
 'jill_scott',
 'janet_jackson',
 'kenny_rogers',
 'smokey_robinson',
 'kylie_minogue',
 'cher',
 'jessie_j',
 'carrie_underwood',
 'nicole_scherzinger',
 'charlotte_gainsbourg',
 'lorde',
 'miley_cyrus',
 'amanda_seyfried',
 'meat_loaf',
 'nelly_furtado',
 'sammy_davis_jr.',
 'cyndi_lauper',
 'anne_hathaway',
 'stevie_wonder',
 'marie_osmond',
 'blake_shelton',
 'bruno_mars',
 'lea_salonga',
 'rita_ora',
 'josh_groban',
 'jack_black',
 'chris_martin']

In [16]:
#jukemeta = full metadata for juke
#voxmeta = full metadata for voxceleb

###getting jukemeta information
for artist in commons:
    print(artist + ": "+ str(len((jukemeta[jukemeta['artist_name'] == artist]))))

troye_sivan: 26
lea_michele: 47
jill_scott: 10
janet_jackson: 8
kenny_rogers: 126
smokey_robinson: 14
kylie_minogue: 60
cher: 143
jessie_j: 7
carrie_underwood: 30
nicole_scherzinger: 43
charlotte_gainsbourg: 5
lorde: 119
miley_cyrus: 56
amanda_seyfried: 27
meat_loaf: 47
nelly_furtado: 9
sammy_davis_jr.: 14
cyndi_lauper: 13
anne_hathaway: 8
stevie_wonder: 167
marie_osmond: 7
blake_shelton: 5
bruno_mars: 213
lea_salonga: 89
rita_ora: 24
josh_groban: 11
jack_black: 118
chris_martin: 4


### TRAIN, TEST, AUXILIARY in JukeBox

In [112]:
jukemeta = pd.read_csv("./jukebox_metadata.csv")
jukemeta['artist_name'] = jukemeta['artist_name'].str.casefold()
jukemeta['artist_name'] = jukemeta['artist_name'].apply(lambda x: x.replace(" ", "_"))

In [113]:
commondf = jukemeta[jukemeta['artist_name'].isin(commons)]
auxiliarys = list(commondf[commondf['partition'] == "AUXILIARY"]["artist_name"].unique())
auxiliary_ID_artist = {}

for aux in auxiliarys:
    idNum = commondf[commondf['artist_name']==aux]['artist_id'].unique()[0]
    auxiliary_ID_artist[str(idNum)] = aux 

In [114]:
aux_ids = [auxid for auxid, name in auxiliary_ID_artist.items()]
aux_ids

['75', '372', '649', '663', '680', '914']

In [115]:
# with open("./auxiliarys.json", 'w+') as auxis:
#     json.dump(auxiliary_ID_artist, auxis)

In [117]:
for artist in auxiliarys:
    print(artist + ": "+ str(len((jukemeta[jukemeta['artist_name'] == artist]))))

charlotte_gainsbourg: 5
anne_hathaway: 8
jack_black: 118
janet_jackson: 8
jessie_j: 7
nelly_furtado: 9


### arranging the data

The data is arranged using a script in `/netscratch/rsharma/voice-recognition-speak-sing/src/utilities.py`. I know I am not an organized being in this sense OKAY! Stop looking at me like a criminal! 

## Meta data for the data arranged in the previous step.

In [28]:
##### Metadata for files and the data size per artists for voxceleb and Jukebox
def get_id_from_artistName(dic, val):
    return [k for k, v in dic.items() if v == val][0]

df_columns = ['artist_name', 'voxceleb_id', 'jukebox_id', 'singing_time_sec', 'speaking_time_sec', 'vox_path', 'juke_path']

In [46]:
import wave
import os

def getDuration(filePath):
    with wave.open(filePath) as mywav:
        duration_seconds = mywav.getnframes() / mywav.getframerate()
    return duration_seconds

speakingDir = "/netscratch/rsharma/voice-recognition-speak-sing/data/speaking/"
singingDir = "/netscratch/rsharma/voice-recognition-speak-sing/data/singing/"
metadata = []


for artist in commons:
    if artist in auxiliarys:
        continue
    voxID = get_id_from_artistName(vox, artist)
    jukeID = get_id_from_artistName(juke, artist)
    
    #singing_metadata
    print("Metadata for: " + artist)
    juke_path = singingDir + artist
    sing_artist_files = [juke_path + "/" + i for i in  os.listdir(singingDir + artist)]
    sing_duration = 0
    for singFile in sing_artist_files:
        sing_duration = duration + getDuration(singFile)
    
    print("Sing duration: " + str(sing_duration))

    
    #speaking_metadata
    vox_path = speakingDir + artist
    speak_artist_folders = [vox_path + "/" + j for j in os.listdir(speakingDir + artist)]
    speak_duration = 0
    for folder in speak_artist_folders:
        speak_artist_files = [folder + "/" + k for k in os.listdir(folder)]
        
        for speakFile in speak_artist_files:
            speak_duration += getDuration(speakFile)
    
    print("Speak duration: " + str(speak_duration))
            
    
    row = [artist, voxID, jukeID, str(int(sing_duration)), str(int(speak_duration)), vox_path, juke_path]
    metadata.append(row)

Metadata for: marie_osmond
Sing duration: 239.76
Speak duration: 690.4849375000001
Metadata for: lea_salonga
Sing duration: 239.76
Speak duration: 1957.6966874999998
Metadata for: bruno_mars
Sing duration: 239.76
Speak duration: 611.2457499999999
Metadata for: smokey_robinson
Sing duration: 239.01
Speak duration: 2447.694937500002
Metadata for: miley_cyrus
Sing duration: 239.76
Speak duration: 2945.8579375000004
Metadata for: amanda_seyfried
Sing duration: 239.76
Speak duration: 1132.1305000000002
Metadata for: josh_groban
Sing duration: 239.28
Speak duration: 2555.0171875000015
Metadata for: nicole_scherzinger
Sing duration: 239.76
Speak duration: 1164.6494374999997
Metadata for: rita_ora
Sing duration: 239.76
Speak duration: 933.5259374999999
Metadata for: cyndi_lauper
Sing duration: 239.76
Speak duration: 1311.6532499999992
Metadata for: stevie_wonder
Sing duration: 239.76
Speak duration: 525.0450000000001
Metadata for: troye_sivan
Sing duration: 239.76
Speak duration: 761.605375
Me

In [3]:
metadata_df = pd.DataFrame(data=metadata, columns=df_columns)
metadata_df.to_csv("/netscratch/rsharma/voice-recognition-speak-sing/data/metdata.csv")

NameError: name 'df_columns' is not defined

# Common data between VoxCeleb2 and JukeBox1

In [17]:
jukeboxIdNameDict = id_name_dict

In [18]:
with open("/netscratch/rsharma/voice-recognition-speak-sing/vox2_id_celeb_pair.json","r") as v2:
    vox2IdNameDict = json.load(v2)

In [19]:
vox2_arts = [j for i,j in vox2IdNameDict.items()]

In [20]:
jukeboxVox2Commons = list(set(jukebox_arts).intersection(set(vox2_arts)))

In [21]:
jukeTestIds = list(jukemeta[jukemeta["partition"] == "TEST"].artist_id.unique())

In [22]:
jukeVox2CommonIds = jukemeta[jukemeta["artist_name"].isin(jukeboxVox2Commons)].artist_id.unique()
jukeVox2CommonIds

array([  20,   57,   62,   80,   82,   98,  125,  344,  358,  373,  379,
        387,  390,  391,  392,  401,  413,  426,  431,  434,  445,  448,
        449,  460,  470,  471,  473,  510,  515,  528,  535,  550,  567,
        573,  611,  624,  653,  709,  725,  740,  744,  753,  754,  757,
        764,  782,  792,  841,  843,  851,  852,  855,  876,  878,  889,
        901,  909,  920,  938,  941,  946,  951,  972,  977, 1002, 1021,
       1029, 1033, 1051, 1059, 1073, 1106, 1112, 1118, 1126, 1127, 1153,
       1177, 1197, 1210, 1232, 1237, 1263, 1271, 1277, 1281, 1294, 1306,
       1314, 1323, 1331, 1336])

In [23]:
jukeVox2CommonsThatAreInTestSetofJuke = set(jukeTestIds).intersection(set(jukeVox2CommonIds))
jukeVox2CommonsThatAreInTestSetofJuke

{401, 431, 510, 851, 876, 1021, 1059, 1073, 1127, 1314}

In [24]:
set(jukeVox2CommonIds).intersection(jukeVox2CommonsThatAreInTestSetofJuke)

{401, 431, 510, 851, 876, 1021, 1059, 1073, 1127, 1314}

### JukeBox artists' ids that are common for both jukebox TRAIN set and voxceleb v2

In [25]:
jukeVox2CommonIds_NoTest = list(set(jukeVox2CommonIds) - jukeVox2CommonsThatAreInTestSetofJuke)
print(jukeVox2CommonIds_NoTest, "\n Number of artists: ", len(jukeVox2CommonIds_NoTest))

[1281, 515, 1029, 1033, 782, 1294, 528, 20, 535, 792, 1306, 1051, 550, 1323, 1331, 567, 1336, 57, 573, 62, 841, 843, 80, 82, 1106, 852, 855, 1112, 344, 1118, 98, 611, 1126, 358, 878, 624, 373, 889, 379, 125, 1153, 387, 901, 390, 391, 392, 653, 909, 920, 1177, 413, 426, 938, 1197, 941, 434, 946, 951, 1210, 445, 448, 449, 709, 460, 972, 1232, 977, 725, 1237, 470, 471, 473, 740, 744, 1002, 1263, 753, 754, 757, 1271, 764, 1277] 
 Number of artists:  82


In [27]:
commonArtistNames= list(jukemeta[jukemeta["artist_id"].isin(jukeVox2CommonIds_NoTest)].artist_name.unique())
print(sorted(commonArtistNames))

#save the id artist pair for jukebox for the common artists in jukebox and vox2
#jukeVox2_id_celeb_commons.json
jV2_id_celeb_commons = {}

for _id in jukeVox2CommonIds_NoTest:
    jV2_id_celeb_commons[int(_id)] = jukemeta[jukemeta["artist_id"] == _id].artist_name.unique()[0]
    
#with open("./jukeVox2_id_celeb_commons.json", "w+") as f:
    
    #json.dump(jV2_id_celeb_commons, f)

['al_jarreau', 'amanda_palmer', 'amel_bent', 'andrea_corr', 'annie_lennox', 'ariana_grande', 'avril_lavigne', 'barbara_cook', 'barbra_streisand', 'barry_manilow', 'billy_idol', 'bob_marley', 'bobby_gillespie', 'bonnie_raitt', 'bret_michaels', 'brian_may', 'bridgit_mendler', 'britney_spears', 'bunga_citra_lestari', 'cara_dillon', 'carla_bruni', 'cerys_matthews', 'chad_kroeger', 'chaka_khan', 'chantal_kreviazuk', 'charlotte_church', 'chester_bennington', 'christophe_maé', 'cinta_laura', 'dany_brillant', 'dave_grohl', 'david_archuleta', 'david_draiman', 'dee_snider', 'demis_roussos', 'dionne_warwick', 'drake', 'emeli_sandé', 'esperanza_spalding', 'garou', 'gavin_rossdale', 'glenn_tilbrook', 'hayley_westenra', 'jack_white', 'john_fogerty', 'jon_bon_jovi', 'joni_mitchell', 'justin_bieber', 'kelis', 'kellie_pickler', 'kelly_rowland', 'keyshia_cole', 'kt_tunstall', 'lara_fabian', 'lemmy', 'marianne_faithfull', 'marilyn_manson', 'martha_wainwright', 'martina_mcbride', 'michael_jackson', 'mike_

In [28]:
#save the id artist pair for voxceleb 2 for the common artists in jukebox and vox2
with open("./vox2_id_celeb_pair.json", "r") as f:
    vox2IdCelebPair = json.load(f)

def filtering_function(pair):
    
    key, value = pair
    if value in commonArtistNames:
        return True
    else:
        return False
    

vox2_id_celeb_commons = dict(filter(filtering_function, vox2IdCelebPair.items()))

#with open("vox2_id_celeb_commons.json", "w+") as f:
    
    #json.dump(vox2_id_celeb_commons, f)

In [29]:
commondf_1 = jukemeta[jukemeta['artist_name'].isin(commonArtistNames)]
auxiliarys_1 = list(commondf_1[commondf_1['partition'] == "AUXILIARY"]["artist_name"].unique())
auxiliary_ID_artist_1 = {}

for aux in auxiliarys_1:
    idNum = commondf_1[commondf_1['artist_name']==aux]['artist_id'].unique()[0]
    auxiliary_ID_artist_1[str(idNum)] = aux 
aux_ids_1 = [auxid for auxid, name in auxiliary_ID_artist_1.items()]
aux_ids_1

['20',
 '82',
 '98',
 '125',
 '373',
 '445',
 '448',
 '471',
 '567',
 '624',
 '653',
 '792',
 '920',
 '1106']

In [30]:
#with open("auxiliarys_jukeVox2.json", "w+") as auxis: 
    #json.dump(auxiliary_ID_artist_1, auxis)

In [31]:
#for artist in auxiliarys_1:
    #print(artist + ": "+ str(len((jukemeta[jukemeta['artist_name'] == artist]))))

In [32]:
with open("jukeVox2_id_celeb_commons.json", 'r') as jk:
    juke1 = json.load(jk)

with open("vox2_id_celeb_commons.json", 'r') as vo:
    vox1 = json.load(vo)

In [33]:
##### Metadata for files and the data size per artists for voxceleb and Jukebox
def get_id_from_artistName(dic, val):
    return [k for k, v in dic.items() if v == val][0]

df_columns = ['artist_name', 'voxceleb2_id', 'jukebox_id', 'singing_time_sec', 'speaking_time_sec', 'vox2_path', 'juke_path']

In [ ]:
import wave
import os

def getDuration(filePath):
    with wave.open(filePath) as mywav:
        duration_seconds = mywav.getnframes() / mywav.getframerate()
    return duration_seconds

speakingDir = "/netscratch/rsharma/voice-recognition-speak-sing/data/speaking/"
singingDir = "/netscratch/rsharma/voice-recognition-speak-sing/data/singing/"
metadata = []


for artist in commonArtistNames:
    if artist in auxiliarys_1:
        continue
    voxID = get_id_from_artistName(vox1, artist)
    jukeID = get_id_from_artistName(juke1, artist)
    
    #singing_metadata
    print("Metadata for: " + artist)
    juke_path = singingDir + artist
    sing_artist_files = [juke_path + "/" + i for i in  os.listdir(singingDir + artist)]
    sing_duration = 0
    for singFile in sing_artist_files:
        sing_duration = duration + getDuration(singFile)
    
    print("Sing duration: " + str(sing_duration))

    
    #speaking_metadata
    vox_path = speakingDir + artist
    speak_artist_folders = [vox_path + "/" + j for j in os.listdir(speakingDir + artist)]
    speak_duration = 0
    for folder in speak_artist_folders:
        speak_artist_files = [folder + "/" + k for k in os.listdir(folder)]
        
        for speakFile in speak_artist_files:
            speak_duration += getDuration(speakFile)
    
    print("Speak duration: " + str(speak_duration))
            
    
    row = [artist, voxID, jukeID, str(int(sing_duration)), str(int(speak_duration)), vox_path, juke_path]
    metadata.append(row)

In [1]:
from src.utilities import Utilities

In [2]:
#Utilities.arrangeVox2_Jukebox1_CommonData()

Arranging vox data for: id00170al_jarreau
Arranging vox data for: id00385amanda_palmer
Arranging vox data for: id00401amel_bent
Arranging vox data for: id00484andrea_corr
Arranging vox data for: id00673annie_lennox
Arranging vox data for: id00752ariana_grande
Arranging vox data for: id00835avril_lavigne
Arranging vox data for: id00890barbara_cook
Arranging vox data for: id00902barbra_streisand
Arranging vox data for: id00907barry_manilow
Arranging vox data for: id01063billy_idol
Arranging vox data for: id01109bob_marley
Arranging vox data for: id01120bobby_gillespie
Arranging vox data for: id01140bonnie_raitt
Arranging vox data for: id01193bret_michaels
Arranging vox data for: id01206brian_may
Arranging vox data for: id01223bridgit_mendler
Arranging vox data for: id01228britney_spears
Arranging vox data for: id01281bunga_citra_lestari
Arranging vox data for: id01347cara_dillon
Arranging vox data for: id01359carla_bruni
Arranging vox data for: id01488cerys_matthews
Arranging vox data fo

In [3]:
Utilities.arrangeJukeV2CommonData()

Arranging Juke data for: 1281dee_snider
Arranging Juke data for: 515david_archuleta
Arranging Juke data for: 1029scott_walker
Arranging Juke data for: 1033sheena_easton
Arranging Juke data for: 782lara_fabian
Arranging Juke data for: 1294chad_kroeger
Arranging Juke data for: 528demis_roussos
Arranging Juke data for: 535dionne_warwick
Arranging Juke data for: 1306vince_neil
Arranging Juke data for: 1051stephen_malkmus
Arranging Juke data for: 550drake
Arranging Juke data for: 1323andrea_corr
Arranging Juke data for: 1331chester_bennington
Arranging Juke data for: 1336david_draiman
Arranging Juke data for: 57bunga_citra_lestari
Arranging Juke data for: 573esperanza_spalding
Arranging Juke data for: 62carla_bruni
Arranging Juke data for: 841marianne_faithfull
Arranging Juke data for: 843marilyn_manson
Arranging Juke data for: 80christophe_maé
Arranging Juke data for: 852martha_wainwright
Arranging Juke data for: 855martina_mcbride
Arranging Juke data for: 1112toby_keith
Arranging Juke dat

In [13]:
import os

singing = os.listdir("/netscratch/rsharma/voice-recognition-speak-sing/data/singing")
speaking = os.listdir("/netscratch/rsharma/voice-recognition-speak-sing/data/speaking")

In [14]:
set(speaking) - set(singing)

set()

In [15]:
set(singing) - set(speaking)

{'britney_spears', 'jon_bon_jovi', 'justin_bieber'}

In [16]:
print(len(singing), len(speaking))

91 88


The remaining three are in the test set. 

In [26]:
dirPath = "/netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/"

In [ ]:
dirpaths = [dirPath + i for i in os.listdir(dirPath)]


Utilities.convertToWav([".m4a"], dirpaths)

Running for /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id00017/
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id00017//iJOjkDq6rW8/00126.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id00017//iJOjkDq6rW8/00123.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id00017//iJOjkDq6rW8/00119.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id00017//iJOjkDq6rW8/00125.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id00017//iJOjkDq6rW8/00129.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id00017//iJOjkDq6rW8/00124.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id00017//iJOjkDq6rW8/00122.m4a
CONVERTING: /netscratch/rsharma/voice-

CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id00017//lvX8diWgriA/00189.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id00017//lvX8diWgriA/00170.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id00017//lvX8diWgriA/00178.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id00017//lvX8diWgriA/00176.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id00017//lvX8diWgriA/00177.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id00017//lvX8diWgriA/00168.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id00017//lvX8diWgriA/00166.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id00017//Nd3m-TBtp6A/00037.m4a
CONVERTING: /net

CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id00017//lZf1RB6l5Gs/00149.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id00017//lZf1RB6l5Gs/00163.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id00017//wrB-yyERH2s/00200.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id00017//wrB-yyERH2s/00201.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id00017//wrB-yyERH2s/00199.m4a
ERROR CONVERTING /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id00017//wrB-yyERH2s/00203.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id00017//wrB-yyERH2s/00202.m4a
ERROR CONVERTING /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id00017//8_a6O3vdlU0/00014.m4a
ERROR 

CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id03969//hBwfRCJ7J68/00074.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id03969//hBwfRCJ7J68/00077.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id03969//hBwfRCJ7J68/00073.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id03969//oMj7ZlZ0Sjc/00096.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id03969//oMj7ZlZ0Sjc/00095.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id03969//oMj7ZlZ0Sjc/00094.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id03969//WZVnB-m0X9g/00039.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id03969//WZVnB-m0X9g/00038.m4a
CONVERTING: /net

CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id03969//zguYH5Ba5V0/00115.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id03969//ozrjmG4qO4M/00097.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id03969//ozrjmG4qO4M/00098.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id03969//ozrjmG4qO4M/00099.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id03969//l3w5_CrHH-0/00087.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id03969//l3w5_CrHH-0/00088.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id03969//l3w5_CrHH-0/00085.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id03969//l3w5_CrHH-0/00086.m4a
CONVERTING: /net

...

CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id08149//O_gtwphGlSg/00021.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id08149//O_gtwphGlSg/00026.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id08149//O_gtwphGlSg/00031.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id08149//O_gtwphGlSg/00024.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id08149//O_gtwphGlSg/00022.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id08149//O_gtwphGlSg/00018.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id08149//O_gtwphGlSg/00019.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id08149//O_gtwphGlSg/00030.m4a
CONVERTING: /net

CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id04232//SKzTcyWk_B8/00220.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id04232//SKzTcyWk_B8/00221.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id04232//SKzTcyWk_B8/00223.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id04232//SKzTcyWk_B8/00224.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id04232//SKzTcyWk_B8/00219.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id04232//SKzTcyWk_B8/00234.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id04232//SKzTcyWk_B8/00232.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id04232//SKzTcyWk_B8/00222.m4a
CONVERTING: /net

CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id04232//p_deCW4NjhI/00368.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id04232//UElg0R7fmlk/00253.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id04232//UElg0R7fmlk/00270.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id04232//UElg0R7fmlk/00258.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id04232//UElg0R7fmlk/00266.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id04232//UElg0R7fmlk/00256.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id04232//UElg0R7fmlk/00259.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id04232//UElg0R7fmlk/00265.m4a
CONVERTING: /net

CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id04232//ljh5PB6Utsc/00357.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id04232//ljh5PB6Utsc/00345.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id04232//ljh5PB6Utsc/00344.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id04232//ljh5PB6Utsc/00356.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id04232//1u5x5RZEAnc/00007.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id04232//1u5x5RZEAnc/00009.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id04232//1u5x5RZEAnc/00011.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id04232//1u5x5RZEAnc/00010.m4a
CONVERTING: /net

CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id04232//NZcKt3gL6F8/00172.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id04232//NZcKt3gL6F8/00181.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id04232//NZcKt3gL6F8/00179.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id04232//NZcKt3gL6F8/00173.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id04232//NZcKt3gL6F8/00174.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id04232//NZcKt3gL6F8/00171.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id04232//NZcKt3gL6F8/00178.m4a
CONVERTING: /netscratch/rsharma/voice-recognition-speak-sing/VoxCeleb_1_2/V2/audio/test/aac/id04232//NZcKt3gL6F8/00176.m4a
CONVERTING: /net

In [25]:
os.listdir(dirPath)

['iJOjkDq6rW8',
 'SFR_KWObD3o',
 'UGu7qDKCDd8',
 'E6aqL_Nc410',
 '5MkXgwdrmJw',
 'tIpW24N_Ysk',
 'M6PYYNz3pac',
 'ABlWCST1MPY',
 'MuKhvYeekXU',
 'blrmdIfXPwI',
 'kQJq9nFET5M',
 'gjYcaCzo7UU',
 'lvX8diWgriA',
 'Nd3m-TBtp6A',
 'X9M79aB1RS4',
 'hcr4tT9y3xs',
 'utfjXffHDgg',
 'b5TvWN3x0HA',
 'QLKR38WeXB0',
 'kN2iEvnTHc4',
 'AXAl9BX9-sQ',
 'yEXiX8Y2qoQ',
 '_mjZ87sK6cA',
 'U7PMtQkYWo8',
 'lZf1RB6l5Gs',
 'wrB-yyERH2s',
 '8_a6O3vdlU0',
 'OLguY5ofUrY',
 'ddLPBVSzuPs',
 '01dfn2spqyE',
 'KtadqxztRBk',
 '7t6lfzvVaTM']